## Analysis number of partient has pair of image L of MLO, CC equal {0, 1}, {1, 0}

In [1]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

### Thống kê lại số lượng trường hợp 0 - 1, 1 - 0 trên đầu người và số lượng trên 

In [2]:
meta = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_data.csv')
meta = meta.drop('Unnamed: 0', axis=1)

In [3]:
meta.head(5)

,patient_id,image_id,view,laterality,age,cancer
0,10006,1874946579,CC,R,61.0,0
1,10006,1864590858,MLO,R,61.0,0
2,10006,462822612,CC,L,61.0,0
3,10006,1459541791,MLO,L,61.0,0
4,10011,541722628,CC,R,55.0,0


In [4]:
print(f"Total image = {len(meta)}")

Total image = 55560


In [5]:
print(f'Total partient id is {len(meta.patient_id.unique())}')

Total partient id is 13507


In [6]:
quanity_cancer = len(meta[meta.cancer == 1].patient_id.unique())
print(f'Quantity partient has cancer: {quanity_cancer}')

Quantity partient has cancer: 2088


In [7]:
print(f"Quantity error feature age:{len(meta[meta.age == 0])}")

Quantity error feature age:80


In [8]:
full_cancer = meta[meta.cancer == 1].patient_id
potential_cancer = meta[meta.patient_id.isin(full_cancer)]
potential_cancer

,patient_id,image_id,view,laterality,age,cancer
80,10130,1165309236,CC,R,71.0,0
81,10130,2110820077,MLO,R,71.0,0
82,10130,613462606,CC,L,71.0,1
83,10130,388811999,MLO,L,71.0,1
84,10130,1360338805,CC,L,71.0,1
...,...,...,...,...,...,...
55555,323CDD,P323-L-DM-MLO,MLO,L,45.0,1
55556,326CDD,P326-R-DM-CC,CC,R,51.0,0
55557,326CDD,P326-R-DM-MLO,MLO,R,51.0,0
55558,326CDD,P326-L-DM-CC,CC,L,51.0,1


In [9]:
group_meta = potential_cancer.groupby((['patient_id', 'laterality','image_id', 'view','cancer'])).size()
# group_meta = group_meta.to_frame()
group_meta

patient_id                        laterality  image_id                          view  cancer
01c3c13f0b852aed00b5147c21d0e649  L           03b3656c726cbe0d79c86a25d4296559  CC    1         1
                                              81bf78c982ca444852fa95af7d96429c  MLO   1         1
                                  R           40a196059322084cbce62642c216b215  MLO   0         1
                                              8c7e6e033f0be620335d5287f9396838  CC    0         1
0246c6050121c176d2e8fcf91f22f926  L           0dbc1663eceef924f0f339604af18364  MLO   1         1
                                                                                               ..
fe23c1647f7617ef219a0a0e07c9eec5  R           a2db9f0fb96f841c6e6da08821e51109  CC    0         1
ff5c656fb6757b708d64d2ebdd33c421  L           11ae77d09d90780bb56ed52bcefb4272  CC    0         1
                                              5c7c3eab43acdb5f4b8ba8b7cdfd5c27  MLO   0         1
                         

In [10]:
def check_cancer(par):
    lat = potential_cancer[potential_cancer.patient_id == par].laterality
    # print(lat)
    for la in lat:
        dic_lat = {'CC': 0, 'MLO': 0}
        # print(la)
        for infor in group_meta[par][la].index:
            dic_lat[infor[-2]] |= infor[-1]
        if dic_lat['CC'] == 1 and dic_lat['MLO'] == 1: return True
    return False
        

In [11]:
number_of_cancer11 = 0
number_of_cancer01 = 0

In [12]:
number_of_cancerL = 0
number_of_cancerR = 0

In [13]:
for par in potential_cancer.patient_id.unique():
    ## Find laterlity

    if(check_cancer(par)): number_of_cancer11 +=1
    else:  
        number_of_cancer01+=1
        nb_1 = potential_cancer[potential_cancer.patient_id == par]
        nb_1 = nb_1[nb_1.cancer == 1].laterality   
        for la in nb_1:
            if la == 'R':
                number_of_cancerR+=1
            else:
                number_of_cancerL+=1
        

In [14]:
meta

,patient_id,image_id,view,laterality,age,cancer
0,10006,1874946579,CC,R,61.0,0
1,10006,1864590858,MLO,R,61.0,0
2,10006,462822612,CC,L,61.0,0
3,10006,1459541791,MLO,L,61.0,0
4,10011,541722628,CC,R,55.0,0
...,...,...,...,...,...,...
55555,323CDD,P323-L-DM-MLO,MLO,L,45.0,1
55556,326CDD,P326-R-DM-CC,CC,R,51.0,0
55557,326CDD,P326-R-DM-MLO,MLO,R,51.0,0
55558,326CDD,P326-L-DM-CC,CC,L,51.0,1


In [15]:
# Find image fake 

In [16]:
full_nocancer = meta[meta.cancer == 0]
full_nocancer

,patient_id,image_id,view,laterality,age,cancer
0,10006,1874946579,CC,R,61.0,0
1,10006,1864590858,MLO,R,61.0,0
2,10006,462822612,CC,L,61.0,0
3,10006,1459541791,MLO,L,61.0,0
4,10011,541722628,CC,R,55.0,0
...,...,...,...,...,...,...
55549,322CDD,P322-R-DM-MLO,MLO,R,76.0,0
55552,323CDD,P323-R-DM-CC,CC,R,45.0,0
55553,323CDD,P323-R-DM-MLO,MLO,R,45.0,0
55556,326CDD,P326-R-DM-CC,CC,R,51.0,0


In [17]:
cnt_fake = 0

In [18]:
# Kiem  tra CC MLO 

In [19]:
img_del = []

In [20]:
import os

In [21]:
## DEL  image fake
views = {'CC': 'MLO',
         'MLO':'CC'
         }
for id_img in meta[meta.cancer == 1].image_id:
    tmp = full_nocancer[full_nocancer.image_id == id_img]
    if(len(tmp) ==1):
        # print(tmp.view.iloc[0])
        # # path1 = f'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_LAT\{tmp.patient_id.iloc[0]}\{tmp.laterality.iloc[0]}\{tmp.view.iloc[0]}\{tmp.view.iloc[0]}_{tmp.laterality.iloc[0]}_{id_img}.png'
        # # path2 = f'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_LAT\{tmp.patient_id.iloc[0]}\{tmp.laterality.iloc[0]}\{views[tmp.view.iloc[0]]}\{views[tmp.view.iloc[0]]}_{tmp.laterality.iloc[0]}_{id_img}.png'
        # # print(path1)
        # # print(path2)
        # # print(path1)
        # # print(id_img)
        # # if len(glob.glob('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_LAT\{tmp.patient_id.iloc[0]}\{tmp.laterality.iloc[0]}\{views[tmp.view.iloc[0]]}\*.png')) ==0 or len(glob.glob('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_LAT\{tmp.patient_id.iloc[0]}\{tmp.laterality.iloc[0]}\{tmp.view.iloc[0]}\{tmp.view.iloc[0]}_{tmp.laterality.iloc[0]}_{id_img}.png'))
        # # os.remove(path1)
        # # os.remove(path2)
        
        cnt_fake+=1
        img_del.append(id_img)
        # break

In [22]:
len(img_del)

238

In [23]:
len(meta)

55560

In [24]:
meta.drop(meta[meta.image_id.isin(img_del)].index, inplace=True)

In [25]:
len(meta)

55084

In [27]:
55560 - 238*2

55084

In [29]:
data = pd.DataFrame({
    
    'number_of_paient11': number_of_cancer11, 
    'number_of_paient01': number_of_cancer01,
    'number_of_cancer_pair_img_L': number_of_cancerL,
    'number_of_cancer_pair_img_R': number_of_cancerR,
    'number_of_img_fake': cnt_fake,
    'number_of_cancer': 4108
    }, index=[1])
data

,number_of_paient11,number_of_paient01,number_of_cancer_pair_img_L,number_of_cancer_pair_img_R,number_of_img_fake,number_of_cancer
1,2053,35,21,14,238,4108


In [28]:
# Số lượng cancer còn lại
len(meta[meta.cancer == 1])

4108

In [30]:
meta.to_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_data.csv')

# Convert format new

In [31]:
# !pip install -qU python-gdcm pydicom pylibjpeg
# !pip install -U pylibjpeg-libjpeg -v
# !pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg
# !pip install pydicom

# import include nesscessaryz
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

In [32]:
from remove_image import *

In [33]:
meta.laterality[0]

'R'

In [34]:
with mp.Pool(12) as p:
    p.map(process_move, list(range(len(meta))))

In [35]:
# Thống kê số lượng

In [37]:
from Evaluate import *

In [38]:
eval = Evaluate(meta, 'Meta')

In [39]:
eval.show()

Dataset Meta 
 Số lượng image: 55084 
 Số luong patient: 13476
So luong CC và MLO
CC     27542
MLO    27542
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             13817
      R             13725
MLO   L             13817
      R             13725
dtype: int64
So luong img cancer: 4108


In [40]:
from sklearn.model_selection import train_test_split


In [46]:
no_cancer_data = meta[meta.cancer == 0]
cancer_data = meta[meta.cancer == 1]


In [47]:
cancer_train, cancer_test_val = train_test_split(cancer_data, test_size=0.3, random_state=42)
cancer_test, cancer_val = train_test_split(cancer_test_val, test_size=0.1, random_state=42)
no_cancer_train, no_cancer_test_val = train_test_split(no_cancer_data, test_size=0.3, random_state=42)
no_cancer_test, no_cancer_val = train_test_split(no_cancer_test_val,  test_size=0.1, random_state=42)

In [48]:
train = pd.concat([cancer_train, no_cancer_train], axis=0).reset_index().drop('index', axis=1)
test = pd.concat([cancer_test, no_cancer_test], axis=0).reset_index().drop('index', axis=1)
val = pd.concat([cancer_val, no_cancer_val], axis=0).reset_index().drop('index', axis=1)

In [51]:
len(train),len(test), len(val)

(38558, 14872, 1654)

In [53]:
train.to_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\train.csv')


In [54]:
test.to_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\test.csv')


In [ ]:
val.to_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\val.csv')
